In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import pkg_resources
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import gymnasium as gym
import stable_baselines3 as sb3
import stable_baselines3.common.logger as logger
import stable_baselines3.common.callbacks as callbacks
import stable_baselines3.common.env_checker as env_checker
from dm_control import mjcf
from dm_control.rl.control import PhysicsError
import imageio
import scipy.spatial
import torch
import torch.nn as nn
import torch.optim as optim
import torch_geometric as pyg
import torch.nn.functional as F
import torch_geometric.nn as gnn
import torch_geometric.loader as pyg_loader
import pytorch_lightning as pl
import torchmetrics
from torch.utils.data import Dataset
from pathlib import Path
from typing import Tuple, Callable, Optional, List, Union
from tqdm import trange
from dm_control.rl.control import PhysicsError
from PIL import Image

from flygym.arena.mujoco_arena import FlatTerrain
from flygym.envs.nmf_mujoco import NeuroMechFlyMuJoCo, MuJoCoParameters
from flygym.state import stretched_pose
import flygym.util.vision as vision
import flygym.util.config as config
from flygym.arena import BaseArena
from flygym.arena.mujoco_arena import OdorArena, FlatTerrain, GappedTerrain, BlocksTerrain
from flygym.util.data import color_cycle_rgb

from rl_navigation import ObstacleOdorArena, NMFNavigation

pygame 2.5.1 (SDL 2.28.2, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/lightning_fabric/__init__.py:36: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning_fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__("pkg_resources").declare_namespace(__name__)
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site

Sanity check on MDP task

In [3]:
terrain_arena = FlatTerrain(ground_alpha=1)
arena = ObstacleOdorArena(
    terrain=terrain_arena,
    obstacle_positions=np.array([(7.5, 0)]),
    obstacle_radius=1,
    odor_source=np.array([[15, 0, 2]]),
    marker_size=0.5,
    obstacle_colors=(0, 0, 0, 1),
)
sim = NMFNavigation(
    arena=arena,
    test_mode=True,
    debug_mode=True,
    decision_dt=0.1,
)
# env_checker.check_env(sim)

In [4]:
sim.reset()
for i in trange(30):
    obs, reward, terminated, truncated, info = sim.step(np.array([0]))
    if terminated:
        print("Terminated")
        break
sim.save_video("test.mp4", stabilization_time=0)

/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


resetting environment


  3%|▎         | 1/30 [00:02<01:25,  2.93s/it]

fly_pos: [0.41921986 0.01079297], final reward=-0.3275081047246944, state=seeking
  dist rew=0.419216, danger=0.373, action diff=0.000, tgt rew=0.000
  dist=14.581


  7%|▋         | 2/30 [00:05<01:15,  2.70s/it]

...

fly_pos: [2.67471707 0.43902927], final reward=0.03371325453725804, state=seeking
  dist rew=1.254639, danger=0.610, action diff=0.000, tgt rew=0.000
  dist=12.333


 13%|█▎        | 4/30 [00:10<01:03,  2.45s/it]

fly_pos: [3.89856741 0.39981143], final reward=0.05213463254444384, state=seeking
  dist rew=1.224470, danger=0.586, action diff=0.000, tgt rew=0.000
  dist=11.109


 13%|█▎        | 4/30 [00:12<01:21,  3.12s/it]

fly_pos: [5.05882077 0.33143416], final reward=-2.6505740777073625, state=collision
  dist rew=1.161927, danger=0.825, action diff=0.000, tgt rew=0.000
  dist=9.947
terminated
Terminated


In [5]:
def make_env():
    terrain_arena = FlatTerrain(ground_alpha=1)
    arena = ObstacleOdorArena(
        terrain=terrain_arena,
        obstacle_positions=np.array([(7.5, 0)]),
        obstacle_radius=1,
        odor_source=np.array([[15, 0, 2]]),
        marker_size=0.5,
        obstacle_colors=(0, 0, 0, 1),
    )
    sim = NMFNavigation(
        arena=arena,
        test_mode=False,
        debug_mode=False,
    )
    return sim

from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env

num_procs = 12
# vec_env = SubprocVecEnv([make_env for i in range(num_procs)])
vec_env = make_vec_env(make_env, n_envs=num_procs, vec_env_cls=SubprocVecEnv)

/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('mpl_toolkits')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/pkg_resources/__init__.py:2871: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('sphinxcontrib')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See 

In [6]:
vec_env.action_space

Box(-1.0, 1.0, (1,), float32)

In [7]:

# terrain_arena = FlatTerrain(ground_alpha=1)
# arena = ObstacleOdorArena(
#     terrain=terrain_arena,
#     obstacle_positions=np.array([(7.5, 1.5), (12.5, -1.5)]),
#     odor_source=np.array([[20, 0, 2]]),
#     marker_size=0.5,
#     obstacle_colors=(0.14, 0.14, 0.2, 1),
# )
# sim = NMFNavigation(
#     arena=arena,
#     test_mode=False,
# )
# env_checker.check_env(sim)

np.random.seed(0)
sb3.common.utils.set_random_seed(0, using_cuda=True)

start_from = None
train = True

log_dir = "logs/trial_21"
checkpoint_callback = callbacks.CheckpointCallback(
    save_freq=100,
    save_path=log_dir,
    name_prefix="trial_21",
    save_replay_buffer=True,
    save_vecnormalize=True,
    verbose=2,
)
my_logger = logger.configure(log_dir, ["tensorboard", "stdout", "csv"])
if start_from is None:
    model = sb3.SAC(
        "MlpPolicy",
        # env=sim,
        env=vec_env,
        policy_kwargs={"net_arch": [16, 16]},
        verbose=2,
        learning_rate=0.01,
    )
else:
    model = sb3.SAC.load(start_from)
    model.set_env(vec_env)
    print(model.verbose, model.learning_rate, model.policy_kwargs)
model.set_logger(my_logger)

if train:
    model.learn(total_timesteps=200_000, progress_bar=True, callback=checkpoint_callback)
    model.save("models/trial_21")

Logging to logs/trial_21


Using cuda device


/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
/home/sibwang/anaconda3/envs/flygym/lib/python3.11/s

Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 19.2     |
|    ep_rew_mean     | -27.5    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 3        |
|    time_elapsed    | 77       |
|    total_timesteps | 240      |
| train/             |          |
|    actor_loss      | 0.274    |
|    critic_loss     | 0.715    |
|    ent_coef        | 0.905    |
|    ent_coef_loss   | -0.169   |
|    learning_rate   | 0.01     |
|    n_updates       | 11       |
---------------------------------


...

Saving model checkpoint to logs/trial_21/trial_21_100800_steps.zip

Saving model replay buffer checkpoint to logs/trial_21/trial_21_replay_buffer_100800_steps.pkl

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12.6     |
|    ep_rew_mean     | -9.73    |
| time/              |          |
|    episodes        | 8096     |
|    fps             | 3        |
|    time_elapsed    | 30173    |
|    total_timesteps | 100824   |
| train/             |          |
|    actor_loss      | 6.43     |
|    critic_loss     | 1.96     |
|    ent_coef        | 0.00983  |
|    ent_coef_loss   | 0.0739   |
|    learning_rate   | 0.01     |
|    n_updates       | 8393     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12.7     |
|    ep_rew_mean     | -9.42    |
| time/              |          |
|    episodes        | 8100     |
|    fps             | 3        |
|    time_elapsed    | 30185    |
|    total_timesteps | 100860   |
| train/             |          |
|    actor_loss      | 6.41     |
|    critic_loss     | 1.56     |
|    ent_coef        | 0.00974  |
|    ent_coef_loss   | -0.0865  |
|    learning_rate   | 0.01     |
|    n_updates       | 8396     |
---------------------------------


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 12.6     |
|    ep_rew_mean     | -9.34    |
| time/              |          |
|    episodes        | 8104     |
|    fps             | 3        |
|    time_elapsed    | 30192    |
|    total_timesteps | 100884   |
| train/             |          |
|    actor_loss      | 6.12     |
|    critic_loss     | 1.96     |
|    ent_coef        | 0.00969  |
|    ent_coef_loss   | -0.0413  |
|    learning_rate   | 0.01     |
|    n_updates       | 8398     |
---------------------------------
